<a href="https://colab.research.google.com/github/liupengzhouyi/LearningPytorch/blob/master/20200325/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Package

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import  Variable
import torch.nn as nn
import torch.nn.functional as F

## Setting Device

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Set Data

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=1)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Setting Classes Name

In [0]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Create Net

In [0]:
# 3*32*32
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # fully connect
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Create Net Model

In [0]:
# our model
net = Net().to(device)

## Show Model

In [7]:
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Create Optimizer

In [0]:
# SGD with momentum
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Create Los Function

In [0]:
criterion = nn.CrossEntropyLoss()

## Train And Test

In [10]:
if __name__ ==  '__main__':
    # Train the network
    for epoch in range(5):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data

            # warp them in Variable
            inputs, labels = Variable(inputs.to(device)), Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = net(inputs)
            # loss
            loss = criterion(outputs, labels)
            # backward
            loss.backward()
            # update weights
            optimizer.step()
            # print statistics
            running_loss += loss.data
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    print("Finished Training")

    print("Beginning Testing")
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        labels = labels.to(device)
        outputs = net(Variable(images.to(device)))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))


[1,  2000] loss: 2.235
[1,  4000] loss: 1.876
[1,  6000] loss: 1.670
[1,  8000] loss: 1.588
[1, 10000] loss: 1.527
[1, 12000] loss: 1.468
[2,  2000] loss: 1.410
[2,  4000] loss: 1.388
[2,  6000] loss: 1.365
[2,  8000] loss: 1.351
[2, 10000] loss: 1.315
[2, 12000] loss: 1.325
[3,  2000] loss: 1.236
[3,  4000] loss: 1.237
[3,  6000] loss: 1.254
[3,  8000] loss: 1.236
[3, 10000] loss: 1.250
[3, 12000] loss: 1.233
[4,  2000] loss: 1.156
[4,  4000] loss: 1.152
[4,  6000] loss: 1.140
[4,  8000] loss: 1.167
[4, 10000] loss: 1.161
[4, 12000] loss: 1.149
[5,  2000] loss: 1.055
[5,  4000] loss: 1.065
[5,  6000] loss: 1.091
[5,  8000] loss: 1.088
[5, 10000] loss: 1.079
[5, 12000] loss: 1.105
Finished Training
Beginning Testing
Accuracy of the network on the 10000 test images: 59 %
